### Imports

In [ ]:
import sys
sys.path.append('../popalign/')
import popalign as PA
import importlib

In [ ]:
PA.__file__

### Load data
Run the first cell if samples are stored individually  
Run the second cell if samples are stored in the same matrix (requires meta data file)

In [ ]:
# Load samples from distinct matrix files
mysamples = {
    'CTRL' : '../data/samples/PBMC.mtx',
    'GMCSF_1ng/ml' : '../data/samples/GMCSF.mtx',
    'IFNG_1ng/ml' : '../data/samples/IFNG.mtx',
    'IL2_10ng/ml' : '../data/samples/IL2.mtx',
    'CD40L_20ng/ml' : '../data/samples/CD40L.mtx',
}
mygenes = '../data/samples/genes.tsv'

pop = PA.load_samples(samples=mysamples, 
                      genes=mygenes,
                      outputfolder='output_samples')

In [ ]:
# Load samples from one matrix file with metadata
# (must include cell_barcode, sample_id columns)
mymatrix = '../data/screen/PBMC-MULT-4/pbmcmult4cd3minus.mtx'
mybarcodes = '../data/screen/PBMC-MULT-4/barcodes.tsv'
mygenes = '../data/screen/PBMC-MULT-4/features.tsv'
mymetadata = '../data/screen/PBMC-MULT-4/meta.csv'

pop = PA.load_multiplexed(matrix=mymatrix, 
                     barcodes=mybarcodes, 
                     metafile=mymetadata, 
                     genes=mygenes,
                     outputfolder='output_multiplexed_test',
                     only=[], # list of sample names to only load the specified samples
                     col=None, # either None or a column name from the meta data
                     value=None) # if col != None, specify value in column to filter samples

### Normalize and gene filter

In [ ]:
# Perform column normalization
# Find best normalization factor
PA.normalize(pop, scaling_factor=None, ncells=None)

In [ ]:
# Plot genes (log cv ~ log cv) and filtering line --use multiple times to find best offset (usually between .7 and 1.5)
PA.plot_gene_filter(pop, offset=1.1)

In [ ]:
# Gene filter the data with the last offset value used in the previous step
PA.filter(pop, remove_ribsomal=True)

In [ ]:
# Remove red blood cells from the data
PA.removeRBC(pop, species='human')

### Dimensionality reduction

In [ ]:
# Generate multiple feature spaces and pick the best one based on reconstruction error
# Run GSEA on each feature
# Generate QC plots
PA.onmf(pop, ncells=5000, nfeats=[5,7], nreps=3, niter=500)

### Build probabilistic models

In [ ]:
# Build a Gaussian Mixture model for each sample
# Type the models subpopulations
importlib.reload(PA)
PA.build_gmms(pop, 
              ks=(4,7), 
              niters=1, 
              training=.8, 
              nreplicates=1,
              reg_covar=True, 
              rendering='grouped', 
              types='defaultpbmc', # either None, 'defaultpbmc' or a dictionary
              figsizegrouped=(20,20),
              figsizesingle=(6,5))

In [ ]:
# If the renderings do not have the right figure size, regenerate the renderings with:
#PA.render_models(pop, figsizegrouped=(20,20), figsizesingle=(6,5), mode='grouped')

### Align subpopulations

In [ ]:
# Align subpopulations of each sample against a reference model's subpopulations
PA.align(pop, ref='CTRL', method='conservative', figsizedeltas=(10,10), figsizeentropy=(10,10))

In [ ]:
# Get list of differentially expressed genes between a reference subpopulation
# and a sample's subpopulation that aligned to it
refcomp = 0
sample = 'GMCSF_1ng/ml'
importlib.reload(PA)

genelist= PA.diffexp(pop,
                      refcomp=refcomp,
                      sample=sample,
                      nbins=20, 
                      cutoff=1,
                      renderhists=True,
                      usefiltered=True)

In [ ]:
importlib.reload(PA)
PA.plot_heatmap(pop,
                refcomp=refcomp,
                genelist=genelist,
                cluster=True, 
                savename='%d_%s' % (refcomp,sample) ,# either None or a string
                figsize=(20,20),
                cmap='Purples',
                samplelimits=False,
                scalegenes=True,
                only=None)

### Visualize genes in subpopulations

In [ ]:
importlib.reload(PA)
PA.plot_genes_gmm_cells(pop,
               sample='unique',
               genelist=genelist,
               savename='',
               metric='correlation',
               method='single',
               clustergenes=False,
               clustercells=False,
               cmap='magma',
               figsize=(15,15)
)

### Rank samples

In [ ]:
# Rank each sample against a reference sample's model
importlib.reload(PA)
PA.rank(pop,
        ref='CTRL', # label of the reference sample
        k=100, # number of cells to randomly sample from sample
        niter=200, # number of iterations
        method='LLR', # LLR for log-likelihood ratio or LL for log-likelihood
        mincells=50, # sample's minimum number of cells to be ranked
        figsize=(10,5)) # plot figure size

### Build unique GMM for all samples

In [ ]:
# Build a unique GMM for the samples concatenated together
importlib.reload(PA)
PA.build_unique_gmm(pop, 
                    ks=(5,20), 
                    niters=3, 
                    training=0.2, 
                    reg_covar=True, 
                    types='defaultpbmc', # either None, 'defaultpbmc' or a dictionary
                    figsize=(6,5))

In [ ]:
# Generate a query plot
PA.plot_query(pop,  pcells=.2, nreps=10, figsize=(5,20), sharey=True)

### 2D visualization

In [ ]:
importlib.reload(PA)
PA.scatter(pop, method='umap', color='CD3D')
PA.scatter(pop, method='umap', color='LYZ')
PA.scatter(pop, method='tsne', color='CD3D')
PA.scatter(pop, method='tsne', color='LYZ')

### 3D visualization

In [ ]:
# Interactive 3D visualization of the data in feature space
importlib.reload(PA)
import plotly
plotly.offline.init_notebook_mode()
PA.plotfeatures(pop)

### Work in progress

In [ ]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#-------------- WORK IN PROGRESS BELOW ----------------
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%